In [6]:
#モジュールを自動リロード
%load_ext autoreload
%autoreload 2

from LLMSearch.RWKVEmbedding import RWKVEmbedding

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import json
with open('settings/settings.json') as f:
    settings = json.load(f)


In [8]:
embedder=RWKVEmbedding(settings)

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading /media/kh/python/2023/RWKV-4b-Pile-3B-20230228-7963.pth ...
Strategy: (total 32+1=33 layers)
* cpu [float32, float32], store 33 layers
0-cpu-float32-float32 1-cpu-float32-float32 2-cpu-float32-float32 3-cpu-float32-float32 4-cpu-float32-float32 5-cpu-float32-float32 6-cpu-float32-float32 7-cpu-float32-float32 8-cpu-float32-float32 9-cpu-float32-float32 10-cpu-float32-float32 11-cpu-float32-float32 12-cpu-float32-float32 13-cpu-float32-float32 14-cpu-float32-float32 15-cpu-float32-float32 16-cpu-float32-float32 17-cpu-float32-float32 18-cpu-float32-float32 19-cpu-float32-float32 20-cpu-float32-float32 21-cpu-float32-float32 22-cpu-float32-float32 23-cpu-float32-float32 24-cpu-float32-float32 25-cpu-float32-float32 26-cpu-float32-float32 27-cpu-float32-float32 28-cpu-float32-float32 29-cpu-float32-float32 30-cpu-float32-float32 31-cpu-float32-float32 32-cpu-float32-float32 
emb.weight                        f32      cpu  50277  2560 


In [32]:
from LLMSearch.TextSplitter import auto_txt_split

glob_path=settings["data_path"] + "/texts/*.txt"
chunk_size_limit= settings["chunk_size_limit"]
auto_txt_split(glob_path, chunk_size_limit,init=True)

UserData/split/ginga_tets.txt_chunk_0.txt
UserData/split/neko.txt_chunk_0.txt


[nltk_data] Downloading package punkt to /home/kh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
import glob
chunk_path_glob=settings["data_path"] + "/split/*.txt"
chunk_path_list=glob.glob(chunk_path_glob)
chunk_path_list

['UserData/split/ginga_tets.txt_chunk_0.txt',
 'UserData/split/ginga_tets.txt_chunk_1.txt',
 'UserData/split/neko.txt_chunk_0.txt']

In [57]:
#chunk_path_listからそれぞれデータを読み込み､RWKVによる埋め込みを行う
import numpy as np
from tqdm import tqdm
import faiss

class TextSearcher:
    def __init__(self,embedder,dim,base_path) -> None:
        self.embedder=embedder
        self.base_path=base_path
        self.path_list=[]
        self.index=faiss.IndexFlatL2(dim)

    def calc_text_file(self,path):
        if path in self.path_list:
            return

        with open(path, 'r') as f:
            text = f.read()
        vec=embedder(text)

        self.index.add(vec.astype('float32'))
        self.path_list.append(path)
        return vec

dim=settings["EMBED_DIM"]
searcher=TextSearcher(embedder,settings["EMBED_DIM"],settings["data_path"])
for path in tqdm(chunk_path_list):
    searcher.calc_text_file(path)

100%|██████████| 3/3 [00:06<00:00,  2.09s/it]


In [55]:
D,I=index.search(vec,1)
I

array([[2]])

In [56]:
faiss.write_index(index, settings["data_path"] + "/meta/index.index")